In [1]:
import argparse
import os

if hasattr(__builtins__,'__IPYTHON__'):
    
    sysName='DY'
    sysfiles='/eos/cms/store/group/phys_egamma/akapoor/ChargeMisID/newsamples2/*.root'
    saveRootFile=False
    outfolder='Output'
    outsuffix='1'
    debugprint=True
    analysis='TTH'
    #branchlist="(ChargeFlip_Category|MyElectron_pt)"
else:
    parser = argparse.ArgumentParser(description='Options for the analyzer')
    parser.add_argument('--name', metavar='n', action='store', type=str,help='short name of the physics process/Data sample', required=True)
    parser.add_argument('--file', metavar='f', action='store', type=str,help='location of files',required=True)
    parser.add_argument('--saveroot',action='store_true', help='save root file',default=False)
    parser.add_argument('--debugprint',action='store_true', help='print analysis settings for debug',default=False)
    parser.add_argument('--outfolder', metavar='of', action='store', type=str,help='location of output',default='Output',required=True)
    parser.add_argument('--outsuffix', metavar='os', action='store', type=str,help='suffix to recognize job',default='Output')
    parser.add_argument('--analysis', metavar='ana', action='store', type=str,help='Folder name that has analysis configs',default='Output',required=True)
    #parser.add_argument('--branchlist', metavar='bl', action='store', type=str,help='branches to store in skim',default="(MyElectron_eta|MyElectron_pt)")
    args = parser.parse_args()
    
    sysName=str(args.name)
    sysfiles=str(args.file)
    saveRootFile=args.saveroot
    outfolder=str(args.outfolder)
    outsuffix=str(args.outsuffix)
    analysis=str(args.analysis)
    debugprint=args.debugprint
    #branchlist=str(args.branchlist)

os.system("mkdir "+outfolder)

256

In [2]:
import ROOT
ROOT.ROOT.EnableImplicitMT()
ROOT.gInterpreter.AddIncludePath("./include/");
ROOT.gInterpreter.ProcessLine('#include "Helper.h"')
from include.PyHelper import *

Welcome to JupyROOT 6.24/00


In [3]:
import yaml
ObjectCutsFile=analysis+"/ObjectCuts.yaml"
DefinitionsFile=analysis+"/Definitions.yaml"
SelectionsFile=analysis+"/Selections.yaml"
BranchesToSaveFile=analysis+"/BranchesToSave.yaml"

ObjectCuts = yaml.safe_load(open(ObjectCutsFile, 'r'))
Definitions = yaml.safe_load(open(DefinitionsFile, 'r'))
Selections = yaml.safe_load(open(SelectionsFile, 'r'))
BranchesToSave = yaml.safe_load(open(BranchesToSaveFile, 'r'))

In [4]:
Ana= MyAnalyzer()
for key in ObjectCuts.keys():
    exec(f'{key}mask=ObjectMask()')
    for cut in ObjectCuts[key]:
        exec(f'{key}mask.addcut("{cut}")')
    exec(f'mask={key}mask.mask')
    exec(f'Ana.definition("{key}",{key}mask.mask)')
for key in Definitions.keys():
    Ana.definition(key,Definitions[key])
for key in Selections.keys():
    Ana.sel(key,Selections[key])
    
Ana.process(sysName,{'Files':sysfiles,'Type':'Bkg','Tree':'Events','Xsec':1})

In [5]:
Ana.prepare()    

Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [6]:
DF=Ana.PD[sysName]['RDF']

In [12]:
import json
if debugprint:
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('Dumping the analysis graph')
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'Object Mask Cuts from {ObjectCutsFile}')
    print(json.dumps(ObjectCuts, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'All the definitions from {DefinitionsFile}')
    print(json.dumps(Definitions, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'All the Event Selections from {SelectionsFile}')
    print(json.dumps(Selections, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'Sample information provided as options')
    print(json.dumps({'Files':sysfiles,'Type':'Bkg','Tree':'Events','Xsec':1}, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print(f'Branches to save taken from {BranchesToSaveFile}')
    print(json.dumps(BranchesToSave, indent=3))
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    if not hasattr(__builtins__,'__IPYTHON__'):
        print('Skimmer options used')
        print(args)
    print('Dumping the cutflow')
    print('---------------------------------------------------')
    print('---------------------------------------------------')
    print('Saving the Action Chart')
    ROOT.RDF.SaveGraph(DF, f'{outfolder}/graph.dot')
    from graphviz import render
    render('dot', 'png', f'{outfolder}/graph.dot')

---------------------------------------------------
---------------------------------------------------
Dumping the analysis graph
---------------------------------------------------
---------------------------------------------------
Object Mask Cuts from TTH/ObjectCuts.yaml
{
   "GoodElectrons": [
      "Electron_pt > 10",
      "Electron_sieie<(0.011+0.019*(abs(Electron_eta+Electron_deltaEtaSC)>1.479))",
      "abs(Electron_dxy)< 0.05",
      "abs(Electron_dz)< 0.1",
      "Electron_mvaTTH > 0.8",
      "Electron_hoe<0.1",
      "Electron_eInvMinusPInv>-0.04",
      "Electron_sip3d<8",
      "Electron_convVeto",
      "Electron_miniPFRelIso_all<0.40",
      "Electron_lostHits==0"
   ]
}
---------------------------------------------------
---------------------------------------------------
All the definitions from TTH/Definitions.yaml
{
   "Nel": "Sum(GoodElectrons)",
   "elpt": "Take(Electron_pt[GoodElectrons], Reverse(Argsort(Electron_pt[GoodElectrons])))",
   "eleta": "Take(Electr

In [8]:
#DF.Display({"event","elpt","eldeepjet"}).Print()

In [9]:
if saveRootFile:
    BRS=BranchesToSave.replace(" ", "")
    if debugprint:
        DF.Snapshot("Events",outfolder+"/output_"+outsuffix+".root",f"{BRS}").Report().Print()
    else:
        DF.Snapshot("Events",outfolder+"/output_"+outsuffix+".root",f"{BRS}")

Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not compatible
Check issue in Varbutindices, size of variables not com

Error: dot: can't open graph.dot


CalledProcessError: Command '['dot', '-Tpng', '-O', 'graph.dot']' returned non-zero exit status 2. [stderr: b"Error: dot: can't open graph.dot\n"]